In [1]:
from openstack import connection, identity
import copy

In [2]:
def connection_to_cluster(project):
    conn = connection.Connection(auth_url="http://controller:35357/v3",
                                 project_name=project,
                                 project_domain_name="default",
                                 user_domain_name="default",
                                 username="admin",
                                 password="QIP@t4Ay")
    return conn

In [3]:
class OSmap(object):
    
    def __init__ (self, connection='', OS=True, with_flavors=True, with_volumes=True, hypervisor_details=False):
        'Инит функция'
        if OS == True:
            self.__conn = connection
            self.__sl = [i for i in self.__conn.compute.servers(all_tenants=True)]
            self.__hl = [i for i in self.__conn.compute.hypervisors()]
            
            if with_flavors == True:
                self.__fd = self.__get_fl_dict()
                self.dict_flavors = copy.deepcopy(self.__fd)
            
            if with_volumes == True:
                self.__vd = self.__get_vl_dict()
                self.dict_volumes = copy.deepcopy(self.__vd)
                
                
            self.dict_hypervisors = self.__get_hp_dict(details=hypervisor_details)
            self.dict_servers = self.__get_vm_dict(flavors=with_flavors)
            self.list_names  = self.__get_names()
    
    
    def get_vm_by_name(self, vmn=''):
        'Функция возвращает объект ВМ на основании имени'
        for i in self.__sl:
            if i.name == vmn:
                return i 
    
    def get_vm_by_id(self, vmn=''):
        'Функция возвращает объект ВМ на основании ID'
        for i in self.__sl:
            if i.id == vmn:
                return i 
    
        
    
    def __get_hp_dict(self, details=False):
        'Функция возвращает словарь гипервизоров'
        d = {}
        for i in self.__hl:
            if details == True:
                d[i.name]=self.__conn.compute.get_hypervisor(i.id).to_dict()
            else:
                d[i.name]=i.to_dict()
        return d   
           
    def __get_fl_dict(self):
        'Функция возвращает словарь флейворов'
        d = {}
        for i in set([i.flavor['id'] for i in self.__sl]):
            d[i]=self.__conn.compute.get_flavor(flavor=i).to_dict()
        return d
    
    def __get_vl_dict(self):
        'Функция возвращает словарь вольюмов'
        d = {}
        for i in [i for i in self.__conn.block_store.volumes(all_tenants=True)]:
            d[i.id]=i.to_dict()
        return d
    
    
    def __get_names (self):
        'Функция возвращает список имен ВМ'
        a = [i.name for i in self.__sl]
        return a
    
            
    def __get_vm_dict(self,flavors=True):
        'Возвращает словарь с описанием ВМ из OS + cpu & ram'
        d = {}
        for i in self.__sl:
            d[i.name] = self.get_vm_by_name(vmn=i.name).to_dict()
            if flavors == True:
                d[i.name]['cpu'] = self.__fd[i.flavor['id']]['vcpus']
                d[i.name]['ram'] = self.__fd[i.flavor['id']]['ram']
        return d
        

In [8]:
conn = connection_to_cluster('pkurp')

In [9]:
x = OSmap(connection_to_cluster('pkurp'), with_flavors=False, with_volumes=False)

In [10]:
len(x.list_names)

1662

In [ ]:
p = conn.identity.projects()
pp = [i for i in p]

In [ ]:
len(pp)

In [ ]:
for i in p: 
    print(i)

In [ ]:
%%time
x = OSmap(connection_to_cluster('admin'), with_flavors=False, with_volumes=False)

In [ ]:
%%time
y = OSmap(connection_to_cluster('admin'), with_flavors=True, with_volumes=False)

In [ ]:
%%time
z = OSmap(connection_to_cluster('admin'), with_flavors=True, with_volumes=True)

In [ ]:
%%time
x1 = OSmap(connection_to_cluster('admin'), with_flavors=False, with_volumes=False, hypervisor_details=True)

In [ ]:
%%time
y1 = OSmap(connection_to_cluster('admin'), with_flavors=True, with_volumes=False, hypervisor_details=True)

In [ ]:
%%time
z1 = OSmap(connection_to_cluster('admin'), with_flavors=True, with_volumes=True, hypervisor_details=True)

In [ ]:
x = z1.dict_volumes

In [ ]:
x

In [ ]:
xl = [d for d in x.keys() if x[d]['volume_type'] == '3par']

In [ ]:
len(xl)

In [ ]:
dl = [[x[d]['id'],x[d]['name'],x[d]['created_at']] for d in xl]

In [ ]:
dls = sorted(dl, key = lambda x: x[2] )

In [ ]:
import openpyxl
from openpyxl import *
ofile='./3par_disks.xlsx'
wb = Workbook ()
try:
    os.remove(ofile)
except OSError:
    pass
ws = wb.active
ws.title=('диски 3par')
ws.cell(column=1,row=1, value='ID')
ws.cell(column=2,row=1, value='NAME')
ws.cell(column=3,row=1, value='DATE')
row=2
for i in dls:
    ws.cell(column=1,row=row, value=i[0])
    ws.cell(column=2,row=row, value=i[1])
    ws.cell(column=3,row=row, value=i[2])
    row = row + 1
wb.save(ofile)
wb.close()

In [ ]:
#z1.list_names

In [ ]:
#vd = z1.dict_volumes
#sd = z1.dict_servers

In [ ]:
#vid = vd['9c5d9579-7a29-4abb-9bd2-05875c84db1d']['attachments'][0]['server_id']

In [ ]:
#z1.get_vm_by_id(vid).name

In [ ]:
#volumes_attached = [i for i in vd.keys() if vd[i]['attachments'] != []]
#volumes_notNone_attached = [i for i in vd.keys() if vd[i]['attachments'] != [] \
#                        and z1.get_vm_by_id(vd[i]['attachments'][0]['server_id']) is not None]
#volumes_dettached = [i for i in vd.keys() if vd[i]['attachments'] == []]
#volumes_None_attached = [i for i in vd.keys() if vd[i]['attachments'] != [] \
                        and z1.get_vm_by_id(vd[i]['attachments'][0]['server_id']) is None]

In [ ]:
#sd[z1.get_vm_by_id(vid).name]
#vd['2d2e33a4-5c0f-48e4-9b1e-1cb10e264803']

In [ ]:
len(vd.keys()), len(volumes_attached), len(volumes_dettached), len(volumes_None_attached), len(volumes_notNone_attached)

In [ ]:
print('Список дисков с кривым аттачем:')
print('')
for v in volumes_None_attached:
    name = vd[v]['name']
    server = z1.get_vm_by_id(vd[v]['attachments'][0]['server_id'])
    size = vd[v]['size']
    vtype = vd[v]['volume_type']
    v_id = vd[v]['id']
    print(server, v_id ,vtype, size, name)
    

In [ ]:
#vd['68e3161d-42c9-4440-be83-25d9652d403a']

In [ ]:
print('Список дисков с аттачем:')
print('')
for v in volumes_notNone_attached:
    name = vd[v]['name']
    server = z1.get_vm_by_id(vd[v]['attachments'][0]['server_id']).name
    size = vd[v]['size']
    vtype = vd[v]['volume_type']
    v_id = vd[v]['id']
    print(server, v_id ,vtype, size, name)

In [ ]:
print('Список дисков без аттача:')
print('')
for v in volumes_dettached:
    name = vd[v]['name']
    #server = z1.get_vm_by_id(vd[v]['attachments'][0]['server_id']).name
    size = vd[v]['size']
    vtype = vd[v]['volume_type']
    v_id = vd[v]['id']
    print(name, v_id ,vtype, size)

In [ ]:
#asd

In [ ]:
target_cn = 'cn-db08.egron.tnx'

In [ ]:
sl = [i for i in z1.dict_servers.keys() if z1.dict_servers[i]['hypervisor_hostname'] == target_cn] 

In [ ]:
sl

In [ ]:
memory_list = [z1.dict_servers[i]['ram'] for i in sl]

In [ ]:
memory = sum(memory_list)

In [ ]:
516900 - memory 

In [ ]:
memory_list